# Regularisierung auf dem Titanic-Datensatz

Hinweis: Da das Notebook nur das Prinizip der Regularisierung zeigen soll, ist der ML-Worflow in diesem Notebook stark vereinfacht (d.h. kein Auffüllen von N/As, kein Feature-Scaling und keine kategorischen Features).

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None  # avoid slide-copy-warning 

In [4]:
df = pd.read_csv("../data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_selection = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]]
df_selection = df_selection.dropna()
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [6]:
df_X = df_selection.drop(columns = ["Survived"])
df_y = df_selection["Survived"]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

## Aufgaben:

### Aufgabe 1: Ridge-Regularisierung
1. Trainieren Sie eine Logistische Regression mit Ridge-Regularisierung (siehe [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html)) mit $\alpha =100$ auf den Trainingsdaten.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [7]:
from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier(alpha=100)
ridge.fit(X_train, y_train)

RidgeClassifier(alpha=100)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = ridge.predict(X_test)
accuracy_score(y_test, predictions)
predictions

0.6713286713286714

In [8]:
from sklearn.metrics import accuracy_score
predictions = ridge.predict(X_test)
predictions

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1])

### Aufgabe 2: Cross-Validation über $\alpha$
1. Führen Sie nun eine Cross-Validation mit Hilfe der Klasse [RidgeClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html) durch. Benutzen Sie dafür die gleichen Werte für $\alpha$ wie in [diesem Notebook](https://github.com/pabair/ml-kurs/blob/master/4_Regularisierung%2BCV.ipynb) in Zelle 9.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [20]:
from sklearn.linear_model import RidgeClassifierCV
alphas = list(range(1, 10000))
ridgecv = RidgeClassifierCV(alphas=alphas, scoring="accuracy")
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

1

In [21]:
predictions = ridgecv.predict(X_test)
accuracy_score(y_test, predictions)

0.6853146853146853

### Aufgabe 3:  Lasso-Regularisierung
Für die Lasso-Regularisierung gibt es keine extra Unterklasse, da diese bei der Klasse `LogisitcRegression` über den Parameter `penalty` eingestellt werden kann. Mit dem Parameter `penalty = "l1"` wird die Lasso-Regression verwendet.
Der Regularisierungsfaktor $\alpha$ wird in diesem Fall über den Parameter `C` bestimmt, welcher den Default-Wert `C=1.0` hat:

`LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")`

Beachten Sie dabei:
- Für die Regression ist das zusätzliche Argument `solver="liblinear"` nötig, da der Standard-Optimierer nicht mit `L1` funktioniert.
- Parameter `C` gibt die <ins>inverse</ins> Stärke der Regularisierung an (Details siehe [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).


1. Trainieren Sie eine Logistische Regression mit Lasso-Regression für verschiedene Parameter `C`:
    - den Default-Wert `C=1.0`.
    - wählen Sie `C` so, dass das Modell sehr stark regularisiert ist.
    - wählen Sie `C` so, dass das Modell keine Regularisierung hat.
2. Vergleiche Sie die Ergebnisse dieser Modelle indem Sie:
    - die Gewichte und deren Summe ausgeben (wie in [diesem Notebook](https://github.com/pabair/ml-kurs/blob/master/4_Regularisierung%2BCV.ipynb)).
    - die Accuracy auf den Testdaten ermitteln. 

In [22]:
# Default-Wert für C: C=1.0
logisticRegr = LogisticRegression(max_iter=1000, penalty="l1", C=1.0, solver="liblinear")
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print("Gewichte der Feature:")
print(pd.Series(logisticRegr.coef_[0], index = X_train.columns))
sum_weights = sum(np.absolute(logisticRegr.coef_[0]))
print("Summe der absoluten Gewichte:", sum_weights)

Accuracy: 0.6923076923076923
Gewichte der Feature:
Pclass   -0.937422
Age      -0.037685
SibSp    -0.308907
Parch     0.298155
Fare      0.007289
dtype: float64
Summe der absoluten Gewichte: 1.5894580836468044


In [23]:
# Starke Regularisierung
logisticRegr = LogisticRegression(max_iter=1000, penalty = "l1", C=1e-10, solver="liblinear")
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print("Gewichte der Feature:")
print(pd.Series(logisticRegr.coef_[0], index = X_train.columns))
sum_weights = sum(np.absolute(logisticRegr.coef_[0]))
print("Summe der absoluten Gewichte:", sum_weights)

Accuracy: 0.5524475524475524
Gewichte der Feature:
Pclass    0.0
Age       0.0
SibSp     0.0
Parch     0.0
Fare      0.0
dtype: float64
Summe der absoluten Gewichte: 0.0


In [24]:
# Keine Regularisierung
logisticRegr = LogisticRegression(max_iter=1000, penalty = "l1", C=10**10, solver="liblinear")
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)
print("Accuracy ist:", accuracy_score(y_test, predictions))
print("Gewichte der Feature:")
print(pd.Series(logisticRegr.coef_[0], index = X_train.columns))
sum_weights = sum(np.absolute(logisticRegr.coef_[0]))
print("Summe der absoluten Gewichte:", sum_weights)

Accuracy ist: 0.6853146853146853
Gewichte der Feature:
Pclass   -1.049621
Age      -0.041660
SibSp    -0.336368
Parch     0.326937
Fare      0.005991
dtype: float64
Summe der absoluten Gewichte: 1.7605768348224387


### Aufgabe 4. Cross-Validierung über  `C`
Ein guter Wert für `C` soll nun mit Hilfe der Methode `cross_val_score` gefunden werden. Ein Beispiel für diese Methode ist:

In [25]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")
cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")

array([0.65217391, 0.73684211, 0.6754386 , 0.71929825, 0.70175439])

Die Methode führt eine k-fold Cross-Validation auf den Trainingsdaten durch und gibt für jeden der k-folds den Score (in diesem Fall "accuracy") des Modell auf den jeweiligen Validierungsdaten zurück.

1. Berechnen Sie den Durchschnitt der Rückgabewerte von cross_val_score um für das gegebene model den durchschnittlichen Accuarcy-Score über alle k-folds zu erhalten.
2. Berechnen Sie für jeden der Werte $ C \in [0.1, 0.2, ...,9.8, 9.9]$ den durchschnittlichen Accuarcy-Score.
3. Trainieren Sie das Modell mit dem besten Wert für $C$ auf allen Trainingsdaten.
4. Bestimmen Sie die Accuracy dieses Modells auf den Testdaten.

In [26]:
np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy"))

0.6988558352402746

In [27]:
C = [i/10 for i in range (1, 100)]
print(C)

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]


In [28]:
max_c = 0
max_score = 0
for c in C:
    model = LogisticRegression(max_iter=1000, penalty = "l1", C=c, solver="liblinear")
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    if np.mean(scores) > max_score:
        max_c = c
        max_score = np.mean(scores)

print(max_c)

0.3


In [29]:
logisticRegr = LogisticRegression(max_iter=1000, penalty = "l1", C=max_c, solver="liblinear")
logisticRegr.fit(X_train, y_train)

LogisticRegression(C=0.3, max_iter=1000, penalty='l1', solver='liblinear')

In [31]:
predictions = logisticRegr.predict(X_test)
print("Accuracy ist:", accuracy_score(y_test, predictions))
print("Gewichte der Feature:")
print(pd.Series(logisticRegr.coef_[0], index = X_train.columns))
display(logisticRegr.coef_)
sum_weights = sum(np.absolute(logisticRegr.coef_[0]))
print("Summe der absoluten Gewichte:", sum_weights)

Accuracy ist: 0.6923076923076923
Gewichte der Feature:
Pclass   -0.674586
Age      -0.028927
SibSp    -0.254142
Parch     0.234209
Fare      0.010811
dtype: float64


array([[-0.6745858 , -0.02892668, -0.25414157,  0.23420914,  0.01081101]])

Summe der absoluten Gewichte: 1.2026741942521533
